In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/Colab\ Notebooks/Side Projects/HP_chat_bot
%ls

/content/drive/MyDrive/Colab Notebooks/Side Projects/HP_chat_bot
 bert_tune_for_context_cn.ipynb
 chinese_database/
 data_preprocessing_for_bert.ipynb
 english_database/
 find_context.ipynb
 fine_tuned_ckiplab_bert_base_chinese/
 harry_potter_chatbot.ipynb
 HP_1_CN.txt
 HP_1_EN.txt
 HP_2_CN.txt
 HP_2_EN.txt
 HP_3_CN.txt
 HP_3_EN.txt
 HP_4_CN.txt
 HP_4_EN.txt
 HP_5_CN.txt
 HP_5_EN.txt
 HP_6_CN.txt
 HP_6_EN.txt
 HP_7_CN.txt
 HP_7_EN.txt
'HP_chatbot_EN&CN.ipynb'
 hp_cn_database/
 HP_CN_QA.csv
 HP_CN_RAG_1.ipynb
 HP_CN_RAG_2.ipynb
 HP_CN_RAG_3.ipynb
 HP_CN_RAG_4.ipynb
 HP_CN_RAG_5.ipynb
 HP_CN_RAG.ipynb
 hp_en_database/
 HP_EN_QA.csv
 HP_EN_RAG.ipynb
 results/
 trained_ckiplab_bert_base_chinese/
'Transformers Training and Inference on Remote Hardware.ipynb'


In [3]:
%pip install langchain-community
%pip install langchain_openai
%pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52

In [4]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [5]:
# Load File
import os
from langchain.document_loaders import TextLoader

# Define the paths for the English Harry Potter book files
english_files = [f'/content/drive/MyDrive/Colab Notebooks/Side Projects/HP_chat_bot/HP_{i}_EN.txt' for i in range(1, 8)]
documents = []                                    # Initialize an empty list to store all documents
for file_path in english_files:                   # Read the content of all English novel files
    if os.path.exists(file_path):                 # Check if the file exists
        loader = TextLoader(file_path)
        documents.extend(loader.load())           # Add each file's document content to the documents list
    else:
    else:
        print(f"File not found: {file_path}")

print(f'Total documents loaded: {len(documents)}')

Total documents loaded: 7


In [6]:
# Split Data Into Chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(     # Initialize the text splitter with specific parameters
    chunk_size=80,
    chunk_overlap=40,
    length_function=len,
    add_start_index=True)
chunks = text_splitter.split_documents(documents)   # Split the documents into smaller chunks

In [7]:
# Set Up OpenAI API Key
import os
API_KEY='YOUR_PERSONAL_OPENAI_API_KEY'
os.environ['OPENAI_API_KEY'] = API_KEY

In [8]:
# Define a function to process data in batches
MAX_BATCH_SIZE = 41666                              # Adjust as needed
def batchify(data, batch_size):
    for i in range(0, len(data), batch_size):
        yield data[i:i + batch_size]

In [9]:
# Data Embedding
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

CHROMA_PATH = 'hp_en_database'                      # Define the path for storing the Chroma database

# Initialize Chroma with the first batch of documents and embeddings
db = Chroma.from_documents(
    documents=chunks[:MAX_BATCH_SIZE],              # Initialize with the first batch to create the Chroma instance
    embedding=OpenAIEmbeddings(model="text-embedding-3-small"),
    persist_directory=CHROMA_PATH
)


# Process the remaining chunks in smaller batches
for batch in batchify(chunks[MAX_BATCH_SIZE:], MAX_BATCH_SIZE):
    db.add_documents(batch)

db.persist()

print(f'Saved {len(chunks)} chunks to {CHROMA_PATH}')

Saved 141098 chunks to hp_en_database


<ipython-input-9-774f83bfcbf7>:19: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [10]:
# Define the prompt template for generating responses
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

In [11]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# Set up the question-answering system
embedding = OpenAIEmbeddings()
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding)

# Get user input for a question
query = input('Question: ')

# Perform similarity search to find relevant context
results = db.similarity_search_with_relevance_scores(query, k=5)

# Check if relevant results were found
if len(results) == 0 or results[0][1] < 0.7:
  print('Unable to find matching results')
else:
  # Prepare the context from the search results
  context = '\n\n---\n\n'.join([doc.page_content for doc, score in results])

  # Create a prompt using the template
  prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
  prompt = prompt_template.format(context=context, question=query)

  # Initialize the ChatOpenAI model
  model = ChatOpenAI()

  # Generate a response using the model
  response = model.predict(prompt)

  # Extract sources from the search results
  sources = [doc.metadata.get("source", None) for doc, _score in results]

  # Format and print the response with sources
  formatted_response = f"Response: {response}\nSources: {sources}"
  print(formatted_response)


<ipython-input-11-640bbff5cc5c>:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding)


Question: What is the name of Albus Dumbledore's phoenix


<ipython-input-11-640bbff5cc5c>:18: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  response = model.predict(prompt)


Response: Fawkes
Sources: ['/content/drive/MyDrive/Colab Notebooks/Side Projects/HP_chat_bot/HP_5_EN.txt', '/content/drive/MyDrive/Colab Notebooks/Side Projects/HP_chat_bot/HP_5_EN.txt', '/content/drive/MyDrive/Colab Notebooks/Side Projects/HP_chat_bot/HP_7_EN.txt', '/content/drive/MyDrive/Colab Notebooks/Side Projects/HP_chat_bot/HP_4_EN.txt', '/content/drive/MyDrive/Colab Notebooks/Side Projects/HP_chat_bot/HP_4_EN.txt']
